In [1]:
import tensorflow as tf

In [2]:
wordEmbeddingDimension = 50
vocabularySize=100
labels=10
filterSizes_paragraph = [3]
filterSizes_allPara=3
paragraphLength=10
num_filters_parargaph=15
num_filters_allPara=20
maxParagraph = 5

filterShapeOfAllPara =[filterSizes_allPara,3,1,num_filters_allPara]
fullyConnectedLayerInput = 150


In [3]:
import numpy as np
class DataParser:
    def __init__(self):
        self.data=[]
        self.maxParagraph=5
        self.maxParagraphLength=10
        
    
    def getDataFromfile(self,fname):
        labels = np.floor(np.random.rand(5)*10)
        labels=labels.astype(int)
        paragraphs = [[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]]
        self.data=[(labels,paragraphs)]
        
        
        labels = np.floor(np.random.rand(5)*10)
        labels=labels.astype(int)
        paragraphs = [[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]]
        
        self.data.append((labels,paragraphs))
        
        self.totalPages = len(self.data)
        
    def nextBatch(self):
        
        label = np.zeros(10,dtype=float)
        data = self.data[0]
        dupPara = np.zeros(self.maxParagraphLength,dtype=int) # [0]*self.maxParagraphLength
        para=[]
        for l in data[0]:
            label[l]=1
        
        for d in data[1]:
            para.append(d)
        
#         print(len(para),self.maxParagraph)
        for i in range(len(para),self.maxParagraph):
            para.append(dupPara)
        
        return (label,para)
n = DataParser()    
n.getDataFromfile("some")
print(n.nextBatch())
print(n.totalPages)

(array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.]), [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])])
2


In [8]:
wordEmbedding = tf.Variable(tf.random_uniform([vocabularySize, wordEmbeddingDimension], -1.0, 1.0))

paragraphList = []
for i in range(maxParagraph):
    paragraphList.append(tf.placeholder(tf.int32,[paragraphLength]))

target = tf.placeholder(tf.float32,[labels])

In [5]:
paragraph=[[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]]


In [6]:
def getParagraphEmbedding(paragraphWords):
    paraEmbedding=tf.nn.embedding_lookup(wordEmbedding,paragraphWords)
    
    return tf.expand_dims(tf.expand_dims(paraEmbedding, -1),0)

In [7]:
paragraphVector = getParagraphEmbedding(paragraphList[0])

In [8]:
paragraphVector

<tf.Tensor 'ExpandDims_1:0' shape=(1, 10, 50, 1) dtype=float32>

In [9]:
def convLayeronParagraph(paragraphVector,filterSizes,num_input_channels,num_filters):
    
    pooled_outputs=[]
    for filter_size in filterSizes:
        shape = [filter_size,wordEmbeddingDimension,1,num_filters]
        
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
        bias= tf.Variable(tf.constant(0.1, shape=[num_filters]))
        conv = tf.nn.conv2d(
                    paragraphVector,
                    weights,
                    strides=[1, 1, wordEmbeddingDimension, 1],
                    padding="SAME",
                    name="conv")
        
        h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
        pool_length=5
        pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, pool_length, 1, 1],
                    strides=[1, pool_length, 1, 1],
                    padding='SAME',
                    name="pool")
        pooled_outputs.append(pooled)
    return tf.reshape(tf.concat(pooled_outputs,axis=0),[1,-1])
#     filter_size = 

In [10]:
# paragraphCNNOutput = convLayeronParagraph (paragraphVector,filterSizes,1,num_filters_parargaph)

In [11]:
# paragraphCNNOutput

In [12]:
def convLayerCombineParagraph(paragraphVectorList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara):
    
    paragraphCNNEmbedding=[]
    
    for paragraph in paragraphVectorList:
        paragraphVector = getParagraphEmbedding(paragraph)
        cnnEmbedding = convLayeronParagraph(paragraphVector,filterSizes_paragraph,1,num_filters_parargaph)
        paragraphCNNEmbedding.append(cnnEmbedding)
    
    allParagraph=tf.expand_dims(tf.expand_dims(tf.concat(paragraphCNNEmbedding,axis=0),-1),0)
    
    shape = filterShapeOfAllPara
    
    weights= tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    bias= tf.Variable(tf.constant(0.1, shape=[num_filters_allPara]))
    
    conv = tf.nn.conv2d(
                    allParagraph,
                    weights,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
    h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
    return tf.reshape(allParagraph,[1,-1])
    

In [13]:
convOutput=convLayerCombineParagraph(paragraphList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara)
# print(convOutput)

In [14]:
def fullyConnectedLayer(convOutput,labels):
    shape = [fullyConnectedLayerInput,labels]
    weights =tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    bias = tf.Variable(tf.constant(0.1, shape=[labels]))
    layer = tf.nn.sigmoid(tf.matmul(convOutput, weights) + bias)
    return layer

In [15]:
prediction=fullyConnectedLayer(convOutput,labels)


In [16]:
cross_entropy = -tf.reduce_sum(((target*tf.log(prediction + 1e-9)) + ((1-target) * tf.log(1 - prediction + 1e-9)) )  , name='xentropy' ) 
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [17]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [6]:
epoch = 10
for e in range(epoch):
    for itr in range(n.totalPages):
        feed_dict_input={}
        data= n.nextBatch()
        feed_dict_input[target]=data[0]
    #     print(target)
    #     print(data[0])
        for p in range(maxParagraph):
            feed_dict_input[paragraphList[p]]= data[1][p]
#         cross_e=session.run(optimizer,feed_dict=feed_dict_input)
#         cross_e=session.run(cross_entropy,feed_dict=feed_dict_input)
#         print(cross_e)

NameError: name 'target' is not defined

In [19]:
target

<tf.Tensor 'Placeholder_5:0' shape=(10,) dtype=float32>

In [20]:
saver = tf.train.Saver()

In [29]:
saver.save(session, 'my_test_model')


 

'my_test_model'

In [27]:
cross_e=session.run(cross_entropy,feed_dict=feed_dict_input)
print(cross_e)
pred=session.run(prediction,feed_dict=feed_dict_input)
print(pred)

4.95892
[[ 0.69964457  0.3772392   0.39140809  0.63380563  0.64570802  0.55700034
   0.20069793  0.19158448  0.33784017  0.49472451]]


In [4]:
session = tf.Session()
new_saver = tf.train.import_meta_graph('my_test_model.meta')
new_saver.restore(session, './my_test_model')


In [31]:
# session.run(tf.global_variables_initializer())
# session.run(tf.all_variables())
# 

Instructions for updating:
Please use tf.global_variables instead.


[array([[-0.4452526 , -0.29423663, -0.50153935, ...,  0.11574361,
         -0.64012015, -0.86587054],
        [ 0.41218293,  0.44474274, -0.28850347, ...,  0.24490707,
         -0.28067708, -0.38763809],
        [ 0.05645598, -0.84165061,  0.03793603, ..., -0.53095514,
          0.13656266, -0.80531859],
        ..., 
        [-0.04811382, -0.65002751, -0.17020655, ..., -0.06298971,
          0.5164938 ,  0.78865671],
        [-0.59353781, -0.24593353, -0.44707608, ...,  0.648736  ,
         -0.88894677, -0.54353166],
        [-0.43917036, -0.65902925, -0.48679495, ...,  0.96385527,
         -0.70145321,  0.48677444]], dtype=float32),
 array([[[[-0.00653904,  0.17389566,  0.10791444, ..., -0.15697493,
           -0.06571897,  0.00452857]],
 
         [[-0.05472038,  0.16013058, -0.04703585, ..., -0.13081908,
           -0.0088509 ,  0.0281056 ]],
 
         [[ 0.15073593, -0.01370054, -0.00497573, ..., -0.15361351,
            0.11164409,  0.05712346]],
 
         ..., 
         [[-0.0

In [7]:
session.run(prediction,feed_dict=feed_dict_input)

NameError: name 'prediction' is not defined

In [10]:
all_vars = tf.trainable_variables()
for v in all_vars:
    print(v)
    print(session.run(v))

Tensor("Variable/read:0", shape=(100, 50), dtype=float32)
[[-0.4452526  -0.29423663 -0.50153935 ...,  0.11574361 -0.64012015
  -0.86587054]
 [ 0.41218293  0.44474274 -0.28850347 ...,  0.24490707 -0.28067708
  -0.38763809]
 [ 0.05645598 -0.84165061  0.03793603 ..., -0.53095514  0.13656266
  -0.80531859]
 ..., 
 [-0.04811382 -0.65002751 -0.17020655 ..., -0.06298971  0.5164938
   0.78865671]
 [-0.59353781 -0.24593353 -0.44707608 ...,  0.648736   -0.88894677
  -0.54353166]
 [-0.43917036 -0.65902925 -0.48679495 ...,  0.96385527 -0.70145321
   0.48677444]]
Tensor("Variable_1/read:0", shape=(3, 50, 1, 15), dtype=float32)
[[[[-0.00653904  0.17389566  0.10791444 ..., -0.15697493 -0.06571897
     0.00452857]]

  [[-0.05472038  0.16013058 -0.04703585 ..., -0.13081908 -0.0088509
     0.0281056 ]]

  [[ 0.15073593 -0.01370054 -0.00497573 ..., -0.15361351  0.11164409
     0.05712346]]

  ..., 
  [[-0.04381448  0.09587254 -0.07994769 ..., -0.0112864  -0.05369294
    -0.04687748]]

  [[-0.01106166  0.

FailedPreconditionError: Attempting to use uninitialized value Variable_15
	 [[Node: _send_Variable_15_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=536905336164673044, tensor_name="Variable_15:0", _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_15)]]

In [11]:
print([v.op.name for v in tf.all_variables()])

Instructions for updating:
Please use tf.global_variables instead.
['Variable', 'Variable_1', 'Variable_2', 'Variable_3', 'Variable_4', 'Variable_5', 'Variable_6', 'Variable_7', 'Variable_8', 'Variable_9', 'Variable_10', 'Variable_11', 'Variable_12', 'Variable_13', 'Variable_14', 'beta1_power', 'beta2_power', 'Variable/Adam', 'Variable/Adam_1', 'Variable_1/Adam', 'Variable_1/Adam_1', 'Variable_2/Adam', 'Variable_2/Adam_1', 'Variable_3/Adam', 'Variable_3/Adam_1', 'Variable_4/Adam', 'Variable_4/Adam_1', 'Variable_5/Adam', 'Variable_5/Adam_1', 'Variable_6/Adam', 'Variable_6/Adam_1', 'Variable_7/Adam', 'Variable_7/Adam_1', 'Variable_8/Adam', 'Variable_8/Adam_1', 'Variable_9/Adam', 'Variable_9/Adam_1', 'Variable_10/Adam', 'Variable_10/Adam_1', 'Variable_13/Adam', 'Variable_13/Adam_1', 'Variable_14/Adam', 'Variable_14/Adam_1', 'Variable_15']
